In [1]:
# Quick and dirty Marcel the Monkey batting projections

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error


In [2]:
# Projecting 2019 from the prior three seasons
proj_yr = 2019
yearly_weights = {2016: 3, 2017: 4, 2018: 5}
prior_yrs = list(yearly_weights.keys())

# I'll project these stats on a per-PA basis
proj_stats = ["ab", "r", "h", "_1b", "_2b", "_3b", "hr", "rbi", "sb", "cs", "bb", "so", "ibb", "hbp", "sh", "sf", "gidp"]

In [3]:
# Load and clean up prior-year data
priors = pd.read_csv('batting.csv').query('yr in @prior_yrs').set_index(['player_id', 'yr'])
priors

pa   ab   r    h  _1b  _2b  _3b  hr  rbi  sb  cs  bb   so  \
player_id yr                                                                 
abadfe01  2016    1    1   0    0    0    0    0   0    0   0   0   0    1   
          2017    0    0   0    0    0    0    0   0    0   0   0   0    0   
abreujo02 2016  695  624  67  183  125   32    1  25  100   0   2  47  125   
          2017  675  621  95  189  107   43    6  33  102   3   0  35  119   
          2018  553  499  68  132   73   36    1  22   78   2   0  37  109   
...             ...  ...  ..  ...  ...  ...  ...  ..  ...  ..  ..  ..  ...   
zuninmi01 2016  192  164  16   34   15    7    0  12   31   0   0  21   65   
          2017  435  387  52   97   47   25    0  25   64   1   0  39  160   
          2018  405  373  37   75   37   18    0  20   44   0   0  24  150   
zychto01  2016    0    0   0    0    0    0    0   0    0   0   0   0    0   
          2017    0    0   0    0    0    0    0   0    0   0   0   0    0   

                ibb  hbp  sh  sf  gidp  
player_id yr                            
abadfe01  2016    0    0   0   0     0  
          2017    0    0   0   0     0  
abreujo02 2016    7   15   0   9    21  
          2017    6   15   0   4    21  
          2018    7   11   0   6    14  
...             ...  ...  ..  ..   ...  
zuninmi01 2016    0    6   0   1     0  
          2017    0    8   0   1     8  
          2018    0    6   0   2     7  
zychto01  2016    0    0   0   0     0  
          2017    0    0   0   0     0  

[4090 rows x 18 columns]

In [4]:
priors_weighted = pd.concat([(priors.query('yr==@yr') * weight) for yr, weight in yearly_weights.items()])
prior_weighted_avg = priors_weighted.groupby('player_id').sum()/sum(yearly_weights.values())
prior_weighted_avg


,pa,ab,r,h,_1b,_2b,_3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
player_id,,,,,,,,,,,,,,,,,,
abadfe01,0.250000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
abreujo02,629.166667,570.916667,76.75,163.750000,97.333333,37.333333,2.666667,26.416667,91.500000,1.833333,0.500000,38.833333,116.333333,6.666667,13.333333,0.000000,6.083333,18.083333
achteaj01,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ackledu01,17.500000,15.250000,1.50,2.250000,2.250000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.250000,0.000000,0.000000,0.000000,0.250000,0.000000
acunaro01,202.916667,180.416667,32.50,52.916667,29.583333,10.833333,1.666667,10.833333,26.666667,6.666667,2.083333,18.750000,51.250000,0.833333,2.500000,0.000000,1.250000,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zimmejo02,4.083333,3.833333,0.00,0.583333,0.583333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.666667,0.000000,0.000000,0.250000,0.000000,0.000000
zimmery01,443.333333,401.416667,58.75,107.916667,61.416667,24.250000,1.083333,21.166667,68.750000,1.750000,0.666667,34.416667,90.916667,1.000000,3.500000,0.000000,4.000000,12.500000
zobribe01,539.750000,465.333333,70.75,127.083333,85.750000,26.083333,3.000000,12.250000,59.833333,3.416667,3.333333,64.916667,69.166667,2.583333,2.500000,2.083333,4.916667,11.916667


In [5]:
# Compute league averages for the prior years
lg_totals = priors.sum()
lg_avg = lg_totals[proj_stats]/lg_totals['pa']
lg_avg

ab      0.894882
r       0.118863
h       0.226188
_1b     0.145577
_2b     0.044901
_3b     0.004532
hr      0.031177
rbi     0.113372
sb      0.013585
cs      0.005214
bb      0.083986
so      0.216788
ibb     0.005102
hbp     0.009616
sh      0.004997
sf      0.006518
gidp    0.019788
dtype: float64

In [6]:
# For any player whose weighted average is under 400, regress to the mean by "filling them in" to 400 PA
# Do this in a function, so I can re-use it later
def regress_players_to_level(df, regress_level):
    pas_to_add = np.where(df['pa'] < 400, 400 - df['pa'], 0)

    # This is a little tricky.  I want to multiply a column (pas_to_add) and a row (regress_level) and get back a df
    # Pandas can't do that, but np.outer() can, and I can convert the output back into a df
    ballast = pd.DataFrame(np.outer(pas_to_add, regress_level))
    ballast.index = df.index
    ballast.columns = proj_stats
    ballast['pa'] = pas_to_add
    cols = ['pa'] + proj_stats
    priors_regressed = df[cols] + ballast[cols]
    return priors_regressed

priors_regressed = regress_players_to_level(prior_weighted_avg, lg_avg)
priors_regressed

,pa,ab,r,h,_1b,_2b,_3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
player_id,,,,,,,,,,,,,,,,,,
abadfe01,400.000000,357.978951,47.515649,90.418485,58.194351,17.949124,1.811842,12.463168,45.320547,5.430483,2.084159,33.573470,86.910804,2.039493,3.844131,1.997709,2.605739,7.910150
abreujo02,629.166667,570.916667,76.750000,163.750000,97.333333,37.333333,2.666667,26.416667,91.500000,1.833333,0.500000,38.833333,116.333333,6.666667,13.333333,0.000000,6.083333,18.083333
achteaj01,400.000000,357.952672,47.545365,90.475032,58.230745,17.960349,1.812975,12.470963,45.348890,5.433880,2.085462,33.594467,86.715001,2.040769,3.846535,1.998958,2.607368,7.915097
ackledu01,400.000000,357.542242,46.965255,88.766749,57.933150,17.174584,1.733658,11.925358,44.364876,5.196147,1.994223,34.124709,85.171219,1.951485,3.678249,1.911504,2.743296,7.568811
acunaro01,400.000000,356.782931,55.925998,97.494469,58.274107,19.682547,2.559935,16.977881,49.010442,9.343984,3.110858,35.302274,93.975203,1.838837,4.395220,0.984903,2.534672,5.566501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zimmejo02,400.000000,358.131905,47.060006,90.134766,58.219640,17.777004,1.794468,12.343655,44.885953,5.378409,2.064173,33.251523,87.496452,2.019936,3.807268,2.228552,2.580751,7.834297
zimmery01,443.333333,401.416667,58.750000,107.916667,61.416667,24.250000,1.083333,21.166667,68.750000,1.750000,0.666667,34.416667,90.916667,1.000000,3.500000,0.000000,4.000000,12.500000
zobribe01,539.750000,465.333333,70.750000,127.083333,85.750000,26.083333,3.000000,12.250000,59.833333,3.416667,3.333333,64.916667,69.166667,2.583333,2.500000,2.083333,4.916667,11.916667


In [7]:
# These are pretty good projections right here.  We could add age adjustments, but I'm stopping here for now
# Let's add some rate stats

def compute_woba(stats):
    woba_weights = {'ob': .702, 'ibb': -.702, 'tb': .37, 'h': -.21}
    return sum([stats[stat]*woba_weights[stat] for stat in woba_weights.keys()])/(stats['pa'] - stats['ibb'])

def add_batting_rate_stats(df):
    df['outs'] = df['ab'] - df['h'] + df['cs'] + df['sf']
    df['tb'] = df['h'] + df['_2b'] + 2*df['_3b'] + 3*df['hr']
    df['ob'] = df['h'] + df['bb'] + df['hbp']
    df['ba'] = df['h'] / df['ab']
    df['obp'] = df['ob'] / (df['pa'] -df['sh'])
    df['slg'] = df['tb'] / df['ab']
    df['ops'] = df['obp'] + df['slg']
    df['woba'] = compute_woba(df)
    return df

projections = add_batting_rate_stats(priors_regressed)

projections.sort_values('woba', ascending=False)[['pa', 'ba', 'obp', 'slg', 'woba']]

,pa,ba,obp,slg,woba
player_id,,,,,
troutmi01,592.166667,0.311230,0.449057,0.605704,0.432177
martijd02,562.666667,0.316983,0.388033,0.624895,0.413588
judgeaa01,457.250000,0.275585,0.399854,0.565663,0.401234
vottojo01,664.500000,0.307525,0.434663,0.508646,0.398751
freemfr01,639.166667,0.306634,0.394915,0.543600,0.388242
...,...,...,...,...,...
mccanja02,414.000000,0.230383,0.284015,0.354754,0.275420
maldoma01,400.000000,0.221136,0.286378,0.359829,0.274213
hechaad01,400.000000,0.247404,0.284555,0.353669,0.268954


In [8]:
# Now let's evaluate the projections, for players who had at least 200 PA in 2019

actuals = pd.read_csv('batting.csv').query('yr == @proj_yr and pa>=200').set_index(['player_id', 'yr'])
actuals = add_batting_rate_stats(actuals)
actuals

,,pa,ab,r,h,_1b,_2b,_3b,hr,rbi,sb,...,sf,gidp,outs,tb,ob,ba,obp,slg,ops,woba
player_id,yr,,,,,,,,,,,,,,,,,,,,,
abreujo02,2019,693,634,85,180,108,38,1,33,123,2,...,10,24,466,319,229,0.283912,0.330447,0.503155,0.833602,0.345689
acunaro01,2019,712,626,127,175,110,22,2,41,101,37,...,1,8,461,324,260,0.279553,0.365169,0.517572,0.882740,0.371246
adamewi01,2019,584,531,69,135,89,25,1,20,52,4,...,1,9,399,222,184,0.254237,0.316695,0.418079,0.734774,0.312617
adamsma01,2019,333,310,42,70,36,14,0,20,56,0,...,1,7,241,144,92,0.225806,0.276276,0.464516,0.740792,0.308620
adriaeh01,2019,234,202,34,55,39,8,3,5,22,0,...,4,2,153,84,81,0.272277,0.349138,0.415842,0.764980,0.324850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wolteto01,2019,411,359,42,94,74,17,2,1,42,0,...,6,9,272,118,138,0.261838,0.337408,0.328691,0.666099,0.288882
wongko01,2019,549,478,61,136,96,25,4,11,59,24,...,5,2,351,202,196,0.284519,0.360958,0.422594,0.783552,0.331364
yastrmi01,2019,411,371,64,101,55,22,3,21,55,2,...,3,4,277,192,137,0.272237,0.334146,0.517520,0.851667,0.354395


In [9]:
comp_stats = ['ba', 'obp', 'slg', 'woba']
comparisons = pd.merge(projections[comp_stats], actuals[comp_stats], on='player_id', how='inner')
comparisons


,ba_x,obp_x,slg_x,woba_x,ba_y,obp_y,slg_y,woba_y
player_id,,,,,,,,
abreujo02,0.286819,0.343179,0.500365,0.350526,0.283912,0.330447,0.503155,0.345689
acunaro01,0.273260,0.343826,0.485535,0.348199,0.279553,0.365169,0.517572,0.371246
adamewi01,0.261155,0.330300,0.413778,0.316168,0.254237,0.316695,0.418079,0.312617
adamsma01,0.253749,0.313547,0.481793,0.330612,0.225806,0.276276,0.464516,0.308620
adriaeh01,0.253933,0.313159,0.395797,0.299371,0.272277,0.349138,0.415842,0.324850
...,...,...,...,...,...,...,...,...
winkeje01,0.273364,0.356773,0.434771,0.337309,0.269231,0.356771,0.473373,0.353037
wolteto01,0.233001,0.319843,0.356558,0.290444,0.261838,0.337408,0.328691,0.288882
wongko01,0.259411,0.345618,0.389070,0.313051,0.284519,0.360958,0.422594,0.331364


In [10]:
# Compute MAEs
for stat in comp_stats:
    error = mean_absolute_error(comparisons[f'{stat}_x'], comparisons[f'{stat}_y'])
    print('MAE', stat, error)


MAE ba 0.0227880351680305
MAE obp 0.022965440266282538
MAE slg 0.051404033558237174
MAE woba 0.02682500190103908


In [11]:
# OK, I've never like regressing to the mean.
# Instead of regressing players with little playing time to the mean, let's regress them to *their* mean
# E.g., the mean of players who don't average 400 PA/season

players_under_400 = prior_weighted_avg.query('pa<400')
regress_level = players_under_400.sum()[proj_stats]/players_under_400['pa'].sum()
regress_level

ab      0.897834
r       0.108381
h       0.210916
_1b     0.138832
_2b     0.041524
_3b     0.004479
hr      0.026081
rbi     0.101531
sb      0.012327
cs      0.004962
bb      0.077494
so      0.242444
ibb     0.003674
hbp     0.009941
sh      0.008812
sf      0.005918
gidp    0.018922
dtype: float64

In [12]:
# This is just as before, but replacing lg_avg with regress_level
priors_regressed_2 = regress_players_to_level(prior_weighted_avg, regress_level)
projections_new = add_batting_rate_stats(priors_regressed_2)

# Let's pick 10 players with <400 PA and see their projections before and after
plyrs = prior_weighted_avg.query('pa<400').sample(10).index
projections.loc[plyrs][comp_stats], projections_new.loc[plyrs][comp_stats]

(                 ba       obp       slg      woba
 player_id                                        
 cuevano01  0.249414  0.312846  0.400341  0.303160
 turnbsp01  0.252169  0.320723  0.416609  0.312618
 mitchbr01  0.249793  0.317430  0.411167  0.308847
 herrmch01  0.239470  0.314286  0.410423  0.308802
 antoldu01  0.252757  0.321396  0.417581  0.313274
 federti01  0.249929  0.316099  0.416639  0.310404
 lombast02  0.250875  0.319243  0.414473  0.311175
 machije01  0.252522  0.321127  0.417192  0.313011
 burnsbi02  0.248404  0.310095  0.390015  0.297595
 ponceda01  0.251273  0.319413  0.416698  0.311671,
                  ba       obp       slg      woba
 player_id                                        
 cuevano01  0.234574  0.295701  0.367687  0.283011
 turnbsp01  0.234372  0.300372  0.377413  0.288697
 mitchbr01  0.232334  0.297409  0.372717  0.285358
 herrmch01  0.228858  0.302228  0.387007  0.294620
 antoldu01  0.234917  0.301004  0.378290  0.289302
 federti01  0.233519  0.297251

In [13]:
# Note the OBPs and SLG are mostly down.  The extent will vary based on how much playing time a player had

# OK now to the MAEs:
comparisons_new = pd.merge(projections_new[comp_stats], actuals[comp_stats], on='player_id', how='inner')
for stat in comp_stats:
    error = mean_absolute_error(comparisons_new[f'{stat}_x'], comparisons_new[f'{stat}_y'])
    print('MAE', stat, error)

MAE ba 0.02436793257935354
MAE obp 0.024161411138986982
MAE slg 0.0553931128035446
MAE woba 0.029048858037889393


In [14]:
# OK, that's actually worse!  So maybe regressing to league average isn't perfect, but it's working
# better than regressing to this lower mean.  Of course, there's selective sampling by only
# evaluating the players who got to 200 PA in 2019.